In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import StratifiedKFold
from tqdm.auto import tqdm
import shutil as sh

In [5]:
!ls /home/heye0507/wheat_detection/data

arvalis_1.csv  ethz_1.csv   sample_submission.csv  train.csv
arvalis_2.csv  inrae_1.csv  test		   usask_1.csv
arvalis_3.csv  rres_1.csv   train


In [2]:
df = pd.read_csv('/home/heye0507/wheat_detection/data/train.csv')
bboxs = np.stack(df['bbox'].apply(lambda x: np.fromstring(x[1:-1], sep=',')))
for i, column in enumerate(['x', 'y', 'w', 'h']):
    df[column] = bboxs[:,i]
df.drop(columns=['bbox'], inplace=True)
df['x_center'] = df['x'] + df['w']/2
df['y_center'] = df['y'] + df['h']/2
df['classes'] = 0
df = df[['image_id','x', 'y', 'w', 'h','x_center','y_center','classes','source']]

In [3]:
df.head()

,image_id,x,y,w,h,x_center,y_center,classes,source
0,b6ab77fd7,834.0,222.0,56.0,36.0,862.0,240.0,0,usask_1
1,b6ab77fd7,226.0,548.0,130.0,58.0,291.0,577.0,0,usask_1
2,b6ab77fd7,377.0,504.0,74.0,160.0,414.0,584.0,0,usask_1
3,b6ab77fd7,834.0,95.0,109.0,107.0,888.5,148.5,0,usask_1
4,b6ab77fd7,26.0,144.0,124.0,117.0,88.0,202.5,0,usask_1


In [3]:
index = list(set(df.image_id))

In [4]:
source = 'train'
if True:
    for fold in [2]:
        val_index = index[len(index)*fold//5:len(index)*(fold+1)//5]
        for name,mini in tqdm(df.groupby('image_id')):
            if name in val_index:
                path2save = 'val2017/'
            else:
                path2save = 'train2017/'
            if not os.path.exists('/home/heye0507/convertor/fold{}/labels/'.format(fold)+path2save):
                os.makedirs('/home/heye0507/convertor/fold{}/labels/'.format(fold)+path2save)
            with open('/home/heye0507/convertor/fold{}/labels/'.format(fold)+path2save+name+".txt", 'w+') as f:
                row = mini[['classes','x_center','y_center','w','h']].astype(float).values
                row = row/1024
                row = row.astype(str)
                for j in range(len(row)):
                    text = ' '.join(row[j])
                    f.write(text)
                    f.write("\n")
            if not os.path.exists('/home/heye0507/convertor/fold{}/images/{}'.format(fold,path2save)):
                os.makedirs('/home/heye0507/convertor/fold{}/images/{}'.format(fold,path2save))
            sh.copy("/home/heye0507/wheat_detection/data/{}/{}.jpg".format(source,name),'/home/heye0507/convertor/fold{}/images/{}/{}.jpg'.format(fold,path2save,name))

In [20]:
!ls /home/heye0507

apex  config  convertor  fastai2  wheat_detection  yolov5


SyntaxError: EOL while scanning string literal (<ipython-input-22-8f81149d192d>, line 1)

In [1]:
import torch
import os
from datetime import datetime
import time
import random
import cv2
import pandas as pd
import numpy as np
import albumentations as A
import matplotlib.pyplot as plt
from albumentations.pytorch.transforms import ToTensorV2
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from glob import glob

from pathlib import Path

SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

In [2]:
path = Path('/home/heye0507/wheat_detection/data')

In [3]:
df = pd.read_csv(path/'train.csv')
df.head()

,image_id,width,height,bbox,source
0,b6ab77fd7,1024,1024,"[834.0, 222.0, 56.0, 36.0]",usask_1
1,b6ab77fd7,1024,1024,"[226.0, 548.0, 130.0, 58.0]",usask_1
2,b6ab77fd7,1024,1024,"[377.0, 504.0, 74.0, 160.0]",usask_1
3,b6ab77fd7,1024,1024,"[834.0, 95.0, 109.0, 107.0]",usask_1
4,b6ab77fd7,1024,1024,"[26.0, 144.0, 124.0, 117.0]",usask_1


In [4]:
bboxs = np.stack(df['bbox'].apply(lambda x: np.fromstring(x[1:-1], sep=',')))
for i, column in enumerate(['x', 'y', 'w', 'h']):
    df[column] = bboxs[:,i]
df.drop(columns=['bbox'], inplace=True)

In [3]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

df_folds = df[['image_id']].copy()
df_folds.loc[:, 'bbox_count'] = 1
df_folds = df_folds.groupby('image_id').count()
df_folds.loc[:, 'source'] = df[['image_id', 'source']].groupby('image_id').min()['source']
df_folds.loc[:, 'stratify_group'] = np.char.add(
    df_folds['source'].values.astype(str),
    df_folds['bbox_count'].apply(lambda x: f'_{x // 15}').values.astype(str)
)
df_folds.loc[:, 'fold'] = 0

for fold_number, (train_index, val_index) in enumerate(skf.split(X=df_folds.index, y=df_folds['stratify_group'])):
    df_folds.loc[df_folds.iloc[val_index].index, 'fold'] = fold_number

/opt/conda/envs/fastai2/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


In [4]:
df_folds.head()

,bbox_count,source,stratify_group,fold
image_id,,,,
00333207f,55,arvalis_1,arvalis_1_3,1
005b0d8bb,20,usask_1,usask_1_1,3
006a994f7,25,inrae_1,inrae_1_1,1
00764ad5d,41,inrae_1,inrae_1_2,0
00b5fefed,25,arvalis_3,arvalis_3_1,3


In [4]:
fold_number = 4
fold = 4

In [5]:
source = 'train'
val_index = df_folds[df_folds['fold'] == fold_number].index.values
for name,mini in tqdm(df.groupby('image_id')):
    if name in val_index:
        path2save = 'val2017/'
    else:
        path2save = 'train2017/'
    if not os.path.exists('/home/heye0507/convertor/fold{}/labels/'.format(fold)+path2save):
        os.makedirs('/home/heye0507/convertor/fold{}/labels/'.format(fold)+path2save)
    with open('/home/heye0507/convertor/fold{}/labels/'.format(fold)+path2save+name+".txt", 'w+') as f:
        row = mini[['classes','x_center','y_center','w','h']].astype(float).values
        row = row/1024
        row = row.astype(str)
        for j in range(len(row)):
            text = ' '.join(row[j])
            f.write(text)
            f.write("\n")
    if not os.path.exists('/home/heye0507/convertor/fold{}/images/{}'.format(fold,path2save)):
        os.makedirs('/home/heye0507/convertor/fold{}/images/{}'.format(fold,path2save))
    sh.copy("/home/heye0507/wheat_detection/data/{}/{}.jpg".format(source,name),'/home/heye0507/convertor/fold{}/images/{}/{}.jpg'.format(fold,path2save,name))

In [6]:
fold_number

4